In [7]:
import csv
import sys
import numpy as np
import math
import pandas as pd
import mord as m
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import linear_model
from sklearn import svm
from math import sqrt


import xgboost as xgb

d = pd.read_csv("raw.csv",index_col=False)

C:\Users\Celia\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [8]:
def pega(sentence):
    sent_str = ""
    for i in sentence:
        sent_str += str(i)
    return sent_str
def cat(outfilename, line):
    with open(outfilename, 'w') as outfile:
            outfile.write(line)
            outfile.close()
def cat2(outfilename, line):
    with open(outfilename,'a') as outfile:
            outfile.write(line)
            outfile.close()

In [9]:
codigo =8
prueba_numero =1

f = 10
p = 5

prefix ='C:/Users/Celia/Desktop/4º/TFG/codigo/python/resultados'
path = pega([prefix,'/errorNaive',codigo,prueba_numero,'.txt'])

cat(path,pega(["p: ",p," f: ",f,"\n"]))
errores =pega([prefix,'/rmse',codigo,prueba_numero,'.txt'])
cat(errores,pega(["Rmse mejores",prueba_numero,"\n"]))
#archivo registro de pruebas
n=pega([prefix,'/registro_pruebas',codigo,'.txt'])
cat2(n,pega(['nº',prueba_numero ,' columna_predecir: BDIY_Index_Open ',"p: ",p," f: ",f," columnas: ",len(d.columns)," n: ",3,"\n"]))

propAnormal = 0.25
max_col = 3

In [ ]:

for col in list(d.columns.values):
    #print(col)
    if(col=='BDIY_Index_Open'):# probar solo con euro
        
        present = d[(p):(len(d)-f)][col].reset_index()
        future = d[(f+p):len(d)][col].reset_index()
        # la etiqueta es el % de incremento en f días (saltando los p+1 primeros)

        label1 = future.subtract(present, fill_value=0).reset_index()
        label2 = (label1[col]/present[col]).reset_index()
        label=label2.drop(label2.columns[[0]], axis='columns') 
    
    
      
        


       

        array = [0] * len(label)
        zeros = pd.DataFrame(data=array)

        a = label.values
       
        rmse =sqrt(mean_squared_error(array, a))
        cat2(path,pega([col," ", rmse,"\n"]))
        # hasta aqui columna label y error del naive
        
        rmse_up_to_now = 1
        columns= []
        column_names = []

        n = 0
        puntoFijo = False
        cambios = 0
        # miemtras que no se lleven el máximo de columnas y siga mejorando
        while n < max_col and (not puntoFijo):
            print(pega(["n=", n]))
            ndf = columns
            ini = 0
            fin = len(d.columns)
            fin=20
            
            #recorre todas las columnas
            for i in np.arange(ini, fin):
                print(i)
                #nombre de la columna
                nombres = pega([list(d.columns.values)[i],1])
                #si esa columna no ha sido cogida ya antes
               
                if not (nombres in column_names):
                    pred = i
                    s = np.arange(1, len(d)+1)# filas de la tabla
                    vs = np.arange(0, p-1) # pasados a utilizar es ahasta p-1 porque no incluyepe la funcion arange
                    name = list(d.columns.values)[pred]
                    names = []
                    #para la columna  i con la que quiero hacer la prediccion  en names estan los posibles pasados de esa columna a usar
                    for v in vs:
                        names.append(pega([name, (p-v),"-"]))
                    df=pd.DataFrame(columns=names)
                    # desde el dia p+1(cosas de indices) hasta el final menos futuro --> tabla
                    for i in np.arange(p,len(d)-f):
                        #para cada filla nrwo representa la fila actual
                        nrow=[]
                        #rellenar las columnas de (EuroPasado10, EuroPasado9, EuroPadaso8....)
                        for v in vs:
                            #calcular los incrementos que ha habido  en un marco temporal de 10 dias atras
                            #d[ el dia a predeccir - (p-v= rango de 10 a 1) dias atras]
                            nrow.append(((d.iloc[i-(p-v)+1][pred])-(d.iloc[i-(p-v)][pred]))/ (d.iloc[i-(p-v)][pred]))
                        df.append(nrow)
                        df = df.append(pd.Series(nrow, index=df.columns ), ignore_index=True)
                     
                    
                    
                    # se añade el mejor conjunto de columnas de la iteracion anterior de n si no es vacio
                    if(len(columns)!=0):
                        df=pd.concat([columns, df], axis=1, sort=False)
                    
                    # crear la matriz de datos con label
                    datos = pd.concat([label, df], axis=1, sort=False)
                    
                   
                    
                    
                    #-----------------EVALUACION---------------------------------------
                   
                        
                            
                   
                    
                    trainsize=1500 # tamaño de inicion del conjunto de entrenamiento
                    s =np.arange(trainsize,len(datos)-f)
                   
                    ## nuevo---------------
                   
                    pred = datos[col]
                   
                    data =datos.drop(col, 1)
                    #----------------------------------------
                    predichos = []
                    reales=[]
                    for j in s:
                        train_x = data[1:j][:]
                        train_y = pred[1:j][:]
                        
                        gbr = GradientBoostingRegressor()
                    

                        
                        gbr.fit(np.array(train_x), np.array(train_y))
                        
                        test=data.iloc[[j+f],:]
                        Y_pred = gbr.predict(test)
                        real =pred[j+f]
                        
                        
                        valor_p =Y_pred[0]
                        predichos.append(valor_p)
                        valor_r=real
                        reales.append(valor_r)
                    
                    
                    rmse =sqrt(mean_squared_error(reales, predichos))
                    cambios +=1
                    
                    #si mejora
                    if(not(np.isnan(rmse)) and rmse<rmse_up_to_now):
                        rmse_up_to_now=rmse
                        ndf= df
                        cat2(errores,pega([col, " con ", name," iteracion", n," ", rmse_up_to_now,"\n"]))
                
                        
             
            print(pega(["mejores columnas iteracion: ",n]))
            print(ndf.columns.values)
                  
            # aqui termina la busqueda de la combinacion de pasasdos para las columnas i para predecir la columna col         
            if(cambios>0 or len(columns==0) or not (columns.columns.values ==ndf.columns.values) ) :
               
                columns = ndf
              
                columns_names = columns.columns.values
                
                print("cambio columnas")
                print(column_names)
                n+=1
            else:
                print("vuelta sin cabios")
                puntoFijo = True
                    
        print("Acabado")
        print(rmse_up_to_now)
        print(ndf.columns.values)

n=0
0


C:\Users\Celia\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
mejores columnas iteracion: 0
['GBP_Curncy_Volatility5-' 'GBP_Curncy_Volatility4-'
 'GBP_Curncy_Volatility3-' 'GBP_Curncy_Volatility2-']
cambio columnas
[]
n=1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
mejores columnas iteracion: 1
['GBP_Curncy_Volatility5-' 'GBP_Curncy_Volatility4-'
 'GBP_Curncy_Volatility3-' 'GBP_Curncy_Volatility2-'
 'GBP_Curncy_Volatility5-' 'GBP_Curncy_Volatility4-'
 'GBP_Curncy_Volatility3-' 'GBP_Curncy_Volatility2-']
cambio columnas
[]
n=2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


     EUR_Curncy_Open  EUR_Curncy_Closing  EUR_Curncy_High  EUR_Curncy_Low  \
0             1.5716              1.5714           1.5794          1.5697   
1             1.5769              1.5794           1.5835          1.5766   
2             1.5787              1.5788           1.5818          1.5723   
3             1.5812              1.5812           1.5845          1.5736   
4             1.5782              1.5782           1.5896          1.5778   
5             1.5896              1.5895           1.5909          1.5676   
6             1.5695              1.5706           1.5709          1.5612   
7             1.5666              1.5665           1.5753          1.5621   
8             1.5693              1.5693           1.5728          1.5636   
9             1.5712              1.5711           1.5762          1.5692   
10            1.5712              1.5712           1.5938          1.5693   
11            1.5948              1.5938           1.5971          1.5842   